In [8]:
import tensorflow as tf
#import tensorflow.compat.v1 as tf
import os
import _pickle as cPickle
import numpy as np
#tf.compat.v1.disable_eager_execution()
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior() 
dir="./cifar-10-batches-py/"
print (os.listdir(dir))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [9]:
def load_data(filename):
    '''read data from datafile'''
    with open(filename,'rb') as f:
        data=cPickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data=[]
        all_labels=[]
        for filename in filenames:
            data,labels=load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            
            self._data=np.vstack(all_data)
            self._data=self._data
            self._labels=np.hstack(all_labels)
            #debug
            print(self._data.shape)
            print(self._labels.shape)
            self._num_examples=self._data.shape[0]
            self._need_shuffle=need_shuffle
            self._indicator=0
            if self._need_shuffle:
                self._shuffle_data()
            
    def _shuffle_data(self):
        #[0,1,2,3,4,5]->[5,3,2,4,0,1]
        p=np.random.permutation(self._num_examples)
        self._data=self._data[p]
        self._labels=self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size examples as a batch."""
        end_indicator=self._indicator+batch_size
        if(end_indicator>self._num_examples):
            if(self._need_shuffle):
                self._shuffle_data()
                self._indicator=0
                end_indicator=batch_size
            else:
                raise Exception("have no more example")
        if(end_indicator>self._num_examples):
            raise Exception("batch size is large than all example")
        batch_data=self._data[self._indicator:end_indicator]
        batch_labels=self._labels[self._indicator:end_indicator]
        self._indicator=end_indicator
        return batch_data,batch_labels
train_filenames=[os.path.join(dir,'data_batch_%d' % i)for i in range(1,6)]
test_filenames=[os.path.join(dir,'test_batch')]
train_data=CifarData(train_filenames,True)
test_data=CifarData(test_filenames,False)
#batch_data,batch_labels=train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)

(10000, 3072)
(10000,)
(20000, 3072)
(20000,)
(30000, 3072)
(30000,)
(40000, 3072)
(40000,)
(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [11]:
batch_size=20
tf.reset_default_graph()
x=tf.placeholder(tf.float32,[batch_size,3072])
is_training=tf.placeholder(tf.bool,[])
#None,eg:[0,5,6,3]
y=tf.placeholder(tf.int64,[batch_size])

x_image = tf.reshape(x,[-1,3,32,32])
#32*32
x_image = tf.transpose(x_image,perm=[0,2,3,1])

x_image_arr=tf.split(x_image,num_or_size_splits=batch_size,axis=0)
result_x_image_arr=[]

for x_single_image in x_image_arr:
    #x_single_image:[1,32,32,3]->[32,32,3]
    x_single_image=tf.reshape(x_single_image,[32,32,3])
    data_aug_1=tf.image.random_flip_left_right(x_single_image)
    data_aug_2=tf.image.random_brightness(data_aug_1,max_delta=63)
    data_aug_3=tf.image.random_contrast(data_aug_2,lower=0.2,upper=1.8)
    x_single_image=tf.reshape(data_aug_3,[1,32,32,3])
    result_x_image_arr.append(x_single_image)
result_x_images=tf.concat(result_x_image_arr,axis=0)

normal_result_x_images=result_x_images/127.5-1


'''
def conv_wrapper(inputs,name,output_channel=32,kernel_size=(3,3),activation=tf.nn.relu,padding='same'):
    return tf.layers.conv2d(inputs,output_channel,kernel_size,padding='same',activation=tf.nn.relu,name=name)

def pooling_wrapper(inputs,name):
    return tf.layers.max_pooling2d(inputs,(2,2),(2,2),name=name)
'''


def conv_wrapper(inputs,name,is_training,output_channel=32,kernel_size=(3,3),activation=tf.nn.relu,padding='same'):
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs,output_channel,kernel_size,padding='same',activation=None,name=name+'conv1')
        bn=tf.layers.batch_normalization(conv2d,training=is_training)
        return activation(bn)

def pooling_wrapper(inputs,name):
    return tf.layers.max_pooling2d(inputs,(2,2),(2,2),name=name)

#conv1:神经元图 ，feature_map，输出图像
conv1_1=conv_wrapper(normal_result_x_images,'conv1_1',is_training)
conv1_2=conv_wrapper(conv1_1,'conv1_2',is_training)
conv1_3=conv_wrapper(conv1_2,'conv1_3',is_training)
pooling1=pooling_wrapper(conv1_3,'pool1')

conv2_1=conv_wrapper(pooling1,'conv2_1',is_training)
conv2_2=conv_wrapper(conv2_1,'conv2_2',is_training)
conv2_3=conv_wrapper(conv2_2,'conv2_3',is_training)
pooling2=pooling_wrapper(conv2_3,'pool2')

conv3_1=conv_wrapper(pooling2,'conv3_1',is_training)
conv3_2=conv_wrapper(conv3_1,'conv3_2',is_training)
conv3_3=conv_wrapper(conv3_2,'conv3_3',is_training)
pooling3=pooling_wrapper(conv3_3,'pool3')


#[None,4*4*32]
flatten=tf.layers.flatten(pooling3)
y_=tf.layers.dense(flatten,10)

#mean square loss
'''
#course:1+e^x
#api:e^x/sum(e^x)
#[[0.01,0,02,0,8...,0.03],[]]
p_y=tf.nn.softmax(y_)
#5->[0,0,0,0,0,1,0,0,0,0]
y_one_hot=tf.one_hot(y,10,dtype=tf.float32)
loss=tf.reduce_mean(tf.square(y_one_hot-p_y))
'''

loss=tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#y_ -> sofmax
#y -> onehot
#loss = ylogy_ `

#index
predict=tf.argmax(y_,1)
#[1,0,1,0,0,1]
correct_prediction=tf.equal(predict,y)
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
with tf.name_scope('train_op'):
    train_op=tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).


In [12]:
def variable_summary(var,name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean=tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('mean',mean)
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.histogram('histogram',var)

with tf.name_scope('summary'):
    variable_summary(conv1_1,'conv1_1')
    variable_summary(conv1_2,'conv1_2')
    variable_summary(conv2_1,'conv2_1')
    variable_summary(conv2_2,'conv2_2')
    variable_summary(conv3_1,'conv3_1')
    variable_summary(conv3_2,'conv3_2')
    
#tensorboard
#1.指定面板图像显示的变量
#2.训练过程中吧这些变量计算出来，输出到文件中
#3.文件解析./tensorboard --logdir=dir

loss_summary=tf.summary.scalar('loss',loss)
#'loss':<10,1.1>,<20>
accuracy_summary=tf.summary.scalar('accuracy',accuracy)
inputs_summary=tf.summary.image('inputs_image',result_x_images)
merged_summary=tf.summary.merge_all()
merged_summary_test=tf.summary.merge([loss_summary,accuracy_summary])

LOG_DIR='.'
run_label='run_vgg_tensorboard'
run_dir=os.path.join(LOG_DIR,run_label)
if (not os.path.exists(run_dir)):
    os.mkdir(run_dir)
train_log_dir=os.path.join(run_dir,'train')
test_log_dir=os.path.join(run_dir,'test')
if (not os.path.exists(train_log_dir)):
    os.mkdir(train_log_dir)
if (not os.path.exists(test_log_dir)):
    os.mkdir(test_log_dir)

In [16]:
init=tf.compat.v1.global_variables_initializer()
train_steps=100000
test_steps=100

output_summary_steps=100

'''
g = tf.Graph()
with g.as_default():
    hello = tf.constant('hello')
    sess = tf.compat.v1.Session(graph=g)
'''
#train 100k 78.04%->82.6%->83.4%->85.6%
with tf.compat.v1.Session() as sess:
    a=sess.run(init)
    train_writer=tf.summary.FileWriter(train_log_dir,sess.graph)
    test_writer=tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data,fixed_test_batch_labels=test_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data,batch_labels=train_data.next_batch(batch_size)
        eval_ops=[loss,accuracy,train_op]
        should_output_summary=((i+1)%output_summary_steps==0)
        if(should_output_summary):
            eval_ops.append(merged_summary)
        
        eval_ops_results=sess.run(
            eval_ops,
            feed_dict={
                x:batch_data,y:batch_labels,is_training:True
            })
        loss_val,acc_val=eval_ops_results[0:2]
        if(should_output_summary):
            train_summary_str=eval_ops_results[-1]
            train_writer.add_summary(train_summary_str,i+1)
            test_summary_str=sess.run([merged_summary_test],
                                      feed_dict={
                                          x:fixed_test_batch_data,
                                          y:fixed_test_batch_labels,
                                          is_training:False,
                                      })[0]
            test_writer.add_summary(test_summary_str,i+1)
        
        if (i%100==0):
            print('[Train]Step:%d,loss:%4.5f,acc:%4.5f'\
                  %(i,loss_val,acc_val))
        if ((i+1) % 1000 == 0):
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels,
                        is_training:False
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train]Step:0,loss:3.88883,acc:0.05000
[Train]Step:100,loss:1.93900,acc:0.30000
[Train]Step:200,loss:2.19375,acc:0.25000
[Train]Step:300,loss:2.12665,acc:0.25000
[Train]Step:400,loss:1.66998,acc:0.40000
[Train]Step:500,loss:1.86839,acc:0.25000
[Train]Step:600,loss:1.35548,acc:0.45000
[Train]Step:700,loss:1.89691,acc:0.40000
[Train]Step:800,loss:1.49235,acc:0.50000
[Train]Step:900,loss:1.67014,acc:0.45000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.15700
[Train]Step:1000,loss:1.27446,acc:0.55000
[Train]Step:1100,loss:1.58637,acc:0.35000
[Train]Step:1200,loss:1.31910,acc:0.45000
[Train]Step:1300,loss:1.51298,acc:0.50000
[Train]Step:1400,loss:1.43021,acc:0.55000
[Train]Step:1500,loss:1.59074,acc:0.40000
[Train]Step:1600,loss:0.62408,acc:0.90000
[Train]Step:1700,loss:1.36471,acc:0.55000
[Train]Step:1800,loss:1.17891,acc:0.65000
[Train]Step:1900,loss:1.51779,acc:0.55000
(10000, 3072)
(10000,)
[Test ] Step: 2000, acc: 0.19150
[Train]Step:2000,loss:1.25341,acc:0.45000
[Train]Step:2100,l

[Train]Step:17100,loss:0.36219,acc:0.85000
[Train]Step:17200,loss:0.36287,acc:0.80000
[Train]Step:17300,loss:0.49962,acc:0.90000
[Train]Step:17400,loss:0.33984,acc:0.80000
[Train]Step:17500,loss:0.54879,acc:0.80000
[Train]Step:17600,loss:0.18609,acc:1.00000
[Train]Step:17700,loss:1.04003,acc:0.80000
[Train]Step:17800,loss:0.60684,acc:0.75000
[Train]Step:17900,loss:0.51236,acc:0.80000
(10000, 3072)
(10000,)
[Test ] Step: 18000, acc: 0.18150
[Train]Step:18000,loss:0.64241,acc:0.75000
[Train]Step:18100,loss:0.47634,acc:0.80000
[Train]Step:18200,loss:0.78476,acc:0.75000
[Train]Step:18300,loss:0.33337,acc:0.80000
[Train]Step:18400,loss:0.44812,acc:0.85000
[Train]Step:18500,loss:0.58472,acc:0.75000
[Train]Step:18600,loss:0.75984,acc:0.80000
[Train]Step:18700,loss:0.63021,acc:0.80000
[Train]Step:18800,loss:0.73076,acc:0.70000
[Train]Step:18900,loss:0.69860,acc:0.75000
(10000, 3072)
(10000,)
[Test ] Step: 19000, acc: 0.17650
[Train]Step:19000,loss:1.00995,acc:0.70000
[Train]Step:19100,loss:0.4

[Test ] Step: 34000, acc: 0.18350
[Train]Step:34000,loss:0.34752,acc:0.85000
[Train]Step:34100,loss:0.46503,acc:0.80000
[Train]Step:34200,loss:0.27780,acc:0.95000
[Train]Step:34300,loss:0.45521,acc:0.90000
[Train]Step:34400,loss:0.30548,acc:0.90000
[Train]Step:34500,loss:0.60692,acc:0.75000
[Train]Step:34600,loss:0.43497,acc:0.90000
[Train]Step:34700,loss:0.47179,acc:0.75000
[Train]Step:34800,loss:0.21764,acc:0.95000
[Train]Step:34900,loss:0.81120,acc:0.70000
(10000, 3072)
(10000,)
[Test ] Step: 35000, acc: 0.27950
[Train]Step:35000,loss:0.20234,acc:0.95000
[Train]Step:35100,loss:0.49711,acc:0.80000
[Train]Step:35200,loss:0.65670,acc:0.70000
[Train]Step:35300,loss:0.37320,acc:0.85000
[Train]Step:35400,loss:0.23397,acc:0.95000
[Train]Step:35500,loss:0.31967,acc:0.85000
[Train]Step:35600,loss:0.29813,acc:0.85000
[Train]Step:35700,loss:0.42476,acc:0.80000
[Train]Step:35800,loss:0.56915,acc:0.75000
[Train]Step:35900,loss:0.46122,acc:0.80000
(10000, 3072)
(10000,)
[Test ] Step: 36000, acc: 

[Train]Step:50900,loss:0.14385,acc:0.95000
(10000, 3072)
(10000,)
[Test ] Step: 51000, acc: 0.26850
[Train]Step:51000,loss:0.18364,acc:0.95000
[Train]Step:51100,loss:0.52368,acc:0.80000
[Train]Step:51200,loss:0.37337,acc:0.85000
[Train]Step:51300,loss:0.25317,acc:0.95000
[Train]Step:51400,loss:0.38035,acc:0.90000
[Train]Step:51500,loss:0.65357,acc:0.80000
[Train]Step:51600,loss:0.41172,acc:0.90000
[Train]Step:51700,loss:0.22199,acc:0.90000
[Train]Step:51800,loss:0.60495,acc:0.90000
[Train]Step:51900,loss:0.13165,acc:0.95000
(10000, 3072)
(10000,)
[Test ] Step: 52000, acc: 0.25950
[Train]Step:52000,loss:0.43218,acc:0.85000
[Train]Step:52100,loss:0.57345,acc:0.80000
[Train]Step:52200,loss:0.24566,acc:0.85000
[Train]Step:52300,loss:0.34693,acc:0.95000
[Train]Step:52400,loss:0.51957,acc:0.80000
[Train]Step:52500,loss:0.27275,acc:0.90000
[Train]Step:52600,loss:0.48325,acc:0.75000
[Train]Step:52700,loss:0.43161,acc:0.90000
[Train]Step:52800,loss:0.26990,acc:0.80000
[Train]Step:52900,loss:0.2

[Train]Step:67700,loss:0.42585,acc:0.85000
[Train]Step:67800,loss:0.62009,acc:0.80000
[Train]Step:67900,loss:0.17681,acc:0.95000
(10000, 3072)
(10000,)
[Test ] Step: 68000, acc: 0.21700
[Train]Step:68000,loss:0.31327,acc:0.90000
[Train]Step:68100,loss:0.16220,acc:0.95000
[Train]Step:68200,loss:0.26791,acc:0.85000
[Train]Step:68300,loss:0.12384,acc:0.95000
[Train]Step:68400,loss:0.43214,acc:0.90000
[Train]Step:68500,loss:0.16442,acc:0.95000
[Train]Step:68600,loss:0.15038,acc:0.95000
[Train]Step:68700,loss:0.25508,acc:0.90000
[Train]Step:68800,loss:0.42109,acc:0.80000
[Train]Step:68900,loss:0.14362,acc:0.95000
(10000, 3072)
(10000,)
[Test ] Step: 69000, acc: 0.28950
[Train]Step:69000,loss:0.08168,acc:1.00000
[Train]Step:69100,loss:0.35978,acc:0.95000
[Train]Step:69200,loss:0.35089,acc:0.90000
[Train]Step:69300,loss:0.17511,acc:0.95000
[Train]Step:69400,loss:0.42495,acc:0.85000
[Train]Step:69500,loss:0.45773,acc:0.80000
[Train]Step:69600,loss:0.27246,acc:0.90000
[Train]Step:69700,loss:0.3

[Train]Step:84500,loss:0.14760,acc:0.95000
[Train]Step:84600,loss:0.14927,acc:0.95000
[Train]Step:84700,loss:0.21259,acc:0.90000
[Train]Step:84800,loss:0.16222,acc:0.95000
[Train]Step:84900,loss:0.76029,acc:0.85000
(10000, 3072)
(10000,)
[Test ] Step: 85000, acc: 0.28200
[Train]Step:85000,loss:0.30502,acc:0.85000
[Train]Step:85100,loss:0.22377,acc:0.95000
[Train]Step:85200,loss:0.26726,acc:0.90000
[Train]Step:85300,loss:0.45616,acc:0.80000
[Train]Step:85400,loss:0.27868,acc:0.90000
[Train]Step:85500,loss:0.15267,acc:0.90000
[Train]Step:85600,loss:0.20966,acc:0.95000
[Train]Step:85700,loss:0.06660,acc:1.00000
[Train]Step:85800,loss:0.04820,acc:1.00000
[Train]Step:85900,loss:0.12575,acc:0.95000
(10000, 3072)
(10000,)
[Test ] Step: 86000, acc: 0.31400
[Train]Step:86000,loss:0.49913,acc:0.80000
[Train]Step:86100,loss:0.28871,acc:0.85000
[Train]Step:86200,loss:0.29540,acc:0.90000
[Train]Step:86300,loss:0.62674,acc:0.75000
[Train]Step:86400,loss:0.15011,acc:0.95000
[Train]Step:86500,loss:0.3